# TP Python Master IBM/RPM
## 2ème partie : manipuler des images biomédicales à l'aide des bibliothèques `SimpleITK` et `pydicom`

Février 2026

### Albertine Dubois - <span class="glyphicon glyphicon-envelope"></span> albertine.dubois@cea.fr et Marion Savanier - <span class="glyphicon glyphicon-envelope"></span> marion.savanier@universite-paris-saclay.fr
----

Téléchargez le contenu du TP et installez SimpleITK.

In [ ]:
!git clone https://github.com/marsvn/PythonM2-jour2.git
!pip install SimpleITK

Les données pour ce TP sont hébergées sur Google drive. Pour les récupérer créez un raccourci du fichier partagé data.zip dans votre drive puis exécutez le code suivant.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!unzip /content/drive/MyDrive/data.zip -d /content

# A) Manipuler des images biomédicales
* [ITK](https://itk.org/) ou Insight Segmentation and Registration Toolkit est un système multi-plateforme open-source offrant aux développeurs une suite complète d’outils logiciels pour l’analyse d’images.
* Parmi elles, [SimpleITK](https://simpleitk.org/) est une couche simplifiée construite sur ITK, destinée à faciliter son utilisation dans les langages interprétés.
* SimpleITK est une boîte à outils d’analyse d’images comprenant un grand nombre de composants prenant en charge les opérations de filtrage générales, la segmentation et l’enregistrement des images.
* SimpleITK lui-même est écrit en C++ mais est disponible pour un grand nombre de langages de programmation, y compris Python.

### Conventions SimpleITK et rappels sur l'encodage des images
    
L'accès au contenu d'une image se fait selon un ordre `x,y,z` classique avec `image.GetPixel(x,y,z)` ou `image[x,y,z]`.

Les indices des images commencent à 0.

Par convention, SimpleITK, à l'instar d'ITK, considère que les images sont des objets physiques qui occupent une région close dans un espace physique. Ce concept est illustré par la figure suivante :

<figure>
<center>
<img src="https://github.com/marsvn/PythonM2-jour2/blob/main/data/ImageOriginAndSpacing.png?raw=1">
<!--img src="http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/Python_html/ImageOriginAndSpacing.png" -->
</figure>

Le contenu de chaque pixel est encodé suivant un certain nombre de valeurs (ou niveaux) possibles, qui peuvent être positives, négatives, entières, décimales suivant la modalité d'imagerie considérée. Cet encodage caractérise ce qu'on appelle le **type** d'un pixel.

Le nombre de niveaux possibles est défini par la profondeur de l'encodage c’est-à-dire le nombre de bits utilisés pour encoder la valeur d'un pixel.

En imagerie médicale, la profondeur de l'encodage peut-être égale à :
* $1$ bit ($2^1=2$ niveaux)
* $2$ bits ($2^2=4$ niveaux)
* $4$ bits ($2^4=16$ niveaux)
* $5$ bits ($2^5=32$  niveaux)
* $8$ bits ($2^8=256$  niveaux)
* $16$ bits ($2^{16}=65536$ niveaux)
* $32$ bits ($2^{32}=4294967296$ niveaux)

<figure>
<center>
<img src="https://github.com/marsvn/PythonM2-jour2/blob/main/data/quantification_TPRX.png?raw=1">
</figure>
    
Voici la liste de types d'image que vous pourriez être amenés à rencontrer en médecine nucléaire. D'autres types existent bien sûr.

<table>
  <tr><td>Nomenclature SimpleITK</td><td>Description</td><td>Valeurs permises</td></tr>
  <tr><td>sitkUInt8</td><td>Unsigned 8 bit integer</td><td>0,1,2,...,255</td></tr>
  <tr><td>sitkUInt16</td><td>Unsigned 16 bit integer</td><td>0,1,2,...,65535</td></tr>
  <tr><td>sitkInt16</td><td>Signed 16 bit integer</td><td>-32768,...,0,...,32767</td></tr>
  <tr><td>sitkFloat32</td><td>32 bit float</td><td>-1.7976931348623157e+308,...,0,...,1.7976931348623157e+308</td></tr>
  <tr><td>sitkFloat64</td><td>64 bit float</td><td>-1.7976931348623157e+308,...,0,...,1.7976931348623157e+308</td></tr>
</table>

Au final, les composants suivants sont nécessaires pour définir complètement une image :
1. Le **type** de pixel [défini à la création, pas de valeur par défaut] : pour la liste complète voir au dessus.
2. Les **dimensions** [défini à la création, pas de valeur par défaut] : le nombre de pixels/voxels dans chaque direction. Cela définit les dimensions de l'image.
3. L'**origine** [par défaut 0] : coordonnées du pixel/voxel ayant l'indice (0,0,0) en unité physique (mm).
4. L'**espacement** [par défaut 1] : distance en mm séparant 2 pixels/voxels adjacents dans chaque dimension.
5. **Matrice des directions** [par défaut la matrice identité] : matrice liant les vecteurs directeurs de l'image et les axes des coordonnées physiques.

### 1.1 Accéder aux attributs d'une image SimpleITK

Importez le module `SimpleITK` pour pouvoir l'utiliser dans votre programme.

In [ ]:
import SimpleITK as sitk

Pour ouvrir une image d'un fichier présent sur le disque, il existe une fonction ```ReadImage```.

In [ ]:
# Lecture d'une image CT au format mhd
image_ct = sitk.ReadImage('./data/CT-Thorax.mhd') # Renvoie une image ou plus généralement un objet de type SimpleITK
type(image_ct)

Il est ensuite très simple d'accèder aux principales informations de l'image.

#### 1.1.1 Origine, taille et résolution

In [ ]:
print("Origine de l'image CT (en mm) :", image_ct.GetOrigin(),'\n')

print("Résolution de l'image CT (taille du voxel en mm) :",image_ct.GetSpacing(),'\n')

print("Taille de l'image CT (en nombre de pixels) :",image_ct.GetSize())
print("Nombre de lignes dans l'image CT :",image_ct.GetSize()[0])
print("Nombre de colonnes dans l'image CT :",image_ct.GetSize()[1])
print("Nombre de coupes/plans dans l'image CT :",image_ct.GetSize()[2],'\n')

print("Direction de l'image CT :",image_ct.GetDirection())

#### 1.1.2 Dimensions

In [ ]:
print("Dimension de l'image CT :",image_ct.GetDimension(),'\n')

print("Largeur de l'image CT (en nombre de pixels) :",image_ct.GetWidth())
print("Hauteur de l'image CT (en nombre de pixels) :",image_ct.GetHeight())
print("Profondeur de l'image CT (en nombre de pixels) :",image_ct.GetDepth())

#### 1.1.3 Type des pixels

In [ ]:
print("Nombre d'octets (1 octet = 8 bits) sur lequel sont encodés les pixels de l'image CT :",image_ct.GetPixelIDValue())
print("Type des pixels de l'image CT :",image_ct.GetPixelIDTypeAsString())
print("Nombre de composantes par pixel de l'image CT :",image_ct.GetNumberOfComponentsPerPixel())

### 1.2 Convertir une image SimpleITK en un array NumPy

**<span style='color:red'>Attention, l'ordre d'accès aux indices d'une image SimpleITK et de la même image après conversion NumPy est inversé !</span>**

* SimpleITK : <code>image[x,y,z]</code> **colonnes - lignes - plans**
* NumPy : <code>image_numpy_array[z,y,x]</code> **plans - lignes - colonnes**

Pour une image 2D après conversion en un array NumPy 2D :
* Le premier indice, **i**, sélectionne la ligne
* Le deuxième indice, **j**, sélectionne la colonne

<center>
<img width =200 src="https://github.com/marsvn/PythonM2-jour2/blob/main/data/2d-array.png?raw=1">
</center>

Pour une image 3D après conversion en un array NumPy 3D :
* Le premier indice, **i**, sélectionne le plan ou la coupe
* Le second indice, **j**, sélectionne la ligne
* Le troisième indice, **k**, sélectionne la colonne


<figure>
<center>
<img height =120 src="https://github.com/marsvn/PythonM2-jour2/blob/main/data/3d-array-stack.png?raw=1">
<img height =120 src="https://github.com/marsvn/PythonM2-jour2/blob/main/data/3d-array.png?raw=1">
</figure>

#### 1.2.1 De SimpleITK à NumPy

Pour avoir accès aux fonctions de la bibliothèques NumPy et pouvoir les appliquer aux valeurs contenues dans l'image de travail, il faut récupérer le tableau multidimensionnel (2D ou 3D) associé à cette image. Pour cela, on fait appel aux fonctions `sitk.GetArrayFromImage()` ou `GetArrayViewFromImage()`.

- `GetArrayFromImage()`: returns a copy of the image data. You can then freely modify the data as it has no effect on the original SimpleITK image.
- `GetArrayViewFromImage()`: returns a view on the image data which is useful for display in a memory efficient manner. You cannot modify the data and __the view will be invalid if the original SimpleITK image is deleted__.
    
Après conversion de l'image en un array NumPy, la bibliothèque Matplotlib peut être utilisée pour la visualisation.

Ainsi intégrée au sein d'un environnement Python scientifique, notre image peut être soumise à toutes sortes d'opérations de traitement d'images ultérieures.

In [ ]:
# Conversion d'une image SimpleITK en un array NumPy
arr_image_ct = sitk.GetArrayFromImage(image_ct)
print("Taille de l'image CT (SimpleITK) = ",image_ct.GetSize())
print("Taille du tableau (NumPy) correspondant = ",arr_image_ct.shape," <--- attention à l'ordre des dimensions en x, y et z !\n")

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(arr_image_ct, cmap='gray')

---
**Question 1.** Quel est le problème et comment le résoudre ?

---

En sélectionnant un plan de coupe particulier puisque que la fonction `imshow` ne sait qu'afficher une image 2D...

In [ ]:
plt.imshow(arr_image_ct[415,:,:], cmap='gray')

On peut ruser en créant une fonction qui permet l'affichage d'une image 3D selon ses 3 plans de coupe

In [ ]:
# Home-made function to show a 3D image in three views
def show_image(arr,sx,sy,sz,color):
    fig, ax = plt.subplots(1,3, figsize=(10, 10))
    ax[0].imshow(arr[sz,:,:], cmap=str(color)) #
    ax[0].set_title("Vue axiale (transverse)")
    ax[1].imshow(arr[:,sy,:], cmap=str(color), origin='lower')
    ax[1].set_title("Vue coronale (frontale)")
    ax[2].imshow(arr[:,:,sx], cmap=str(color), origin='lower')
    ax[2].set_title("Vue sagittale")

In [ ]:
# View CT
from ipywidgets import interact, interactive, fixed
interact(show_image, arr=fixed(arr_image_ct), sx=(0,image_ct.GetSize()[0]-1), sy=(0,image_ct.GetSize()[1]-1), sz=(0,image_ct.GetSize()[2]-1),color='gray');

#### 1.2.2 De Numpy à SimpleITK

On utilise la fonction suivante :

- `GetImageFromArray()`: Get a SimpleITK Image from a numpy array. If isVector is True, then the Image will have a Vector pixel type, and the last dimension of the array will be considered the component index. By default when isVector is None, 4D images are automatically considered 3D vector images.

N'oubliez pas de définir l'origine de l'image (`SetOrigin([x,y])`), sa résolution (`SetSpacing([sx,sy,sz])`) et potentiellement sa matrice des directions (`SetDirection()`). Les valeurs par défaut peuvent ne pas correspondre aux dimensions physiques de votre image.

Le plus simple, c'est d'appliquer la méthode `CopyInformation()` à l'image que l'on souhaite enregistrer avec pour paramètre le nom de l'image de référence dont on récupère les informations.

Exemple : <code>new_img.CopyInformation(original_img)</code>   


In [ ]:
import numpy as np

arr = np.zeros((10,20,3))
print('Dimensions du tableau NumPy = ',arr.shape)

# If this is supposed to be a 3D gray scale image [x=3, y=20, z=10]
img = sitk.GetImageFromArray(arr)
print("Taille de l'image 3D en niveaux de gris après conversion SimpleITK =",img.GetSize())
print("Nombre de composantes par pixel de cette image après conversion SimpleITK =",img.GetNumberOfComponentsPerPixel())

# If this is supposed to be a 2D color image [x=20,y=10]
img = sitk.GetImageFromArray(arr, isVector=True)
print("Taille de l'image 2D en couleurs après conversion SimpleITK =",img.GetSize())
print("Nombre de composantes par pixel de cette image après conversion SimpleITK =",img.GetNumberOfComponentsPerPixel())

### 1.3 Opérations basiques sur les images

On peut profiter des fonctionnalités NumPy sur les tableaux pour réaliser des opérations telles que :
1. la symétrie
2. le sous-échantillonnage
3. le recadrage

In [ ]:
arr_200 = arr_image_ct[200,:,:] # Ici on sélectionne le plan de coupe d'indice 200
plt.imshow(arr_200,cmap='gray')

In [ ]:
# Suppression des axes et ajout d'une colorbar
plt.imshow(arr_200,cmap='gray')
plt.axis('off')
plt.colorbar()

#### 1.3.1 Symétrie

In [ ]:
# Vertical flip (reverse the y axis)
arr_200_vflip = arr_200[::-1,::] # [start:stop:step]
# By default, start is 0 (first element), stop is last element index and step is 1.
# A negative step can be used to obtain a reversed list. Negative step changes a way, slice notation works.
# It makes the slice be built from the tail of the list.
# So, it goes from the last element to the first element.
# This way, we reverse the array order along y

# Horizontal flip (reverse the x axis)
arr_200_hflip = arr_200[::,::-1]

plt.subplots(1, 3, figsize=(12, 6))
plt.subplot(1,3,1)
plt.imshow(arr_200,cmap='gray')
plt.axis('off')
plt.title('Image originale')
plt.subplot(1,3,2)
plt.imshow(arr_200_vflip,cmap='gray')
plt.axis('off')
plt.title('Symétrie miroir verticale')
plt.subplot(1,3,3)
plt.imshow(arr_200_hflip,cmap='gray')
plt.axis('off')
plt.title('Symétrie miroir horizontale')

#### 1.3.2 Sous-échantillonnage

In [ ]:
# Subsampling
# Here, we keep only every 4-th element of the array
arr_200_subsample = arr_200[::4,::4]

plt.subplots(1, 2, figsize=(10, 10))
plt.subplot(1,2,1)
plt.imshow(arr_200,cmap='gray')
plt.title('Image originale')
#plt.axis('off');
plt.subplot(1,2,2)
plt.imshow(arr_200_subsample,cmap='gray')
plt.title('Image sous-échantillonnée')
#plt.axis('off');

#### 1.3.3 Recadrage

In [ ]:
# Crop the original CT image so as to extract the heart
heart = arr_200[100:260,180:380]
plt.imshow(heart,cmap='gray')
plt.title('Image du coeur')

## Exercice 1
---

* Charger l'image `1-PT.nii` contenue dans le sous-dossier `data`
* Afficher les dimensions de cette image
* Convertir l'image en un tableau (array) NumPy de la même taille
* Afficher les dimensions de ce tableau
* Identifier un plan de coupe sur lequel on voit nettement la tumeur qui se trouve dans le poumon
* Rogner l'image afin d'effecteur un zoom sur cette tumeur et afficher le résultat


In [ ]:
image_pet = sitk.ReadImage(...)
print("Taille de l'image CT (SimpleITK) = ",...)

In [ ]:
arr_image_pet = sitk.GetArrayFromImage(...)
print("Taille du tableau (NumPy) correspondant = ",...)

In [ ]:
plan = ...
tumeur = arr_image_pet[plan,...:...,...:...]
plt.imshow(tumeur,cmap='...')

## 2. Structure d'un fichier DICOM

L'extension .dcm désigne un fichier DICOM.

Dans un fichier DICOM, les données sont organisées de façon séquentielle en commencant par une **entête** (1%), suivie des **données brutes** de l'image (99%).

<center>
<img width=500 src="https://github.com/marsvn/PythonM2-jour2/blob/main/data/dicom2.png?raw=1">
</center>

L'entête DICOM ou *DICOM Tag Header* contient un ensemble d'informations spécifiques appelées **meta-données**, renseignées par des attributs aussi appelés étiquettes ou *tags* en anglais.

* Attributs constructeur
* Attributs patient (état civil, âge, poids etc.)
* Attributs examen (région explorée, technique d'imagerie utilisée, date d'acquisition, praticien etc.)

Lire l'entête DICOM peut s'avérer utile pour :
* Contrôler l'origine d'une image
* Vérifier des paramètres d'acquisition (traceur, kV, champ magnétique)
* Extraire des données pour la recherche (traitement d'images)

Les données brutes de l'image peuvent être :
* Non comprimées (RAW)
* ...ou comprimées (24 types différents)
    * RLE
    * JPEG Lossless (1:2 sans perte)
    * JPEG lossy (1:20 avec perte)
    * JPEG2000...

A l'issue d'un examen médical, un dossier parfois nommé *DICOMDIR* contient la liste des fichiers générés (images, compressées ou non, parfois des vidéos voire des compte-rendus). Il est possible de visualiser la totalité d'un examen dans l'ordre chronologique et par série (par exemple, coupes avant injection, coupes après injection, reconstruction selon un autre plan de coupe etc.).

<center>
<img width=600 src="https://github.com/marsvn/PythonM2-jour2/blob/main/data/dicomstructure.jpg?raw=1">
</center>
    
Les particularités des fichiers DICOM ainsi que les besoins spécifiques des médecins qui les utilisent justifient l'existence de logiciels dédiés pour les manipuler mais aussi pour les classer et pour les partager en réseau (PACS - Picture Archiving and Communication Systems).

Les consoles d'acquisition et de nombreux logiciels libres de visualisation et de traitement d'images permettent aujourd'hui la lecture des entêtes et des images DICOM. Le site de la NEMA en propose une large sélection, régulièrement mise à jour (http://idoimaging.com/programs).

In [ ]:
# Import des librairies de base
import numpy as np
import matplotlib.pyplot as plt
import os

# Import de la librairie SimpleITK
import SimpleITK as sitk

# Import some interactive IPython widgets
# The interact function (ipywidgets.interact) automatically creates user interface (UI) controls for exploring code and data interactively.
from ipywidgets import interact, interactive, fixed

# Definition du répertoire de travail courant
PWD_DIR = os.getcwd()

# Definition du répertoire dans lequel on stockera les données de sortie
OUTPUT_DIR = '/content/PythonM2-jour2/output'
# S'il n'existe pas, on le crée
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# B) Lecture des en-têtes DICOM avec SIMPLEITK

## Première méthode pour charger des séries d'images DICOM

Dans les cellules suivantes, nous allons procéder à la **lecture** de séries d'images DICOM obtenues après une acquisition TEP/TDM, à leur **affichage** et à leur **sauvegarde** sous la forme de fichiers image 3D au format mhd/mha ou nii. Dans notre exemple, les deux séries d'images sont stockées dans un seul et même répertoire (mais ce n'est pas toujours le cas).

### 1. Lecture des séries d'images DICOM TEP et TDM

#### 1.1 Identification des numéros de séries corresondant aux données TEP et TDM

In [ ]:
# Define DICOM data directory
data_dcm_directory = './data/patient2/'
print("Path to DICOM data directory:",data_dcm_directory)

# Provide a file name that belongs to the DICOM CT series you want to read
#!ls './data/patient2/'
file_name_ct= os.path.join(data_dcm_directory,'Patient2.CT._.4.1.2019.11.21.11.52.34.596.71223801.dcm')

# Read the file's meta-information without reading bulk pixel data
file_reader_ct=sitk.ImageFileReader()
file_reader_ct.SetFileName(file_name_ct)
file_reader_ct.ReadImageInformation()

# Get the sorted file names, open all files in the directory and read the meta-information without reading the bulk pixel data
series_ID_ct=file_reader_ct.GetMetaData('0020|000e')
sorted_file_names_ct=sitk.ImageSeriesReader.GetGDCMSeriesFileNames(data_dcm_directory,series_ID_ct)
print("Series ID corresponding to CT data:",series_ID_ct)

# Provide a file name that belongs to the DICOM PET series you want to read
file_name_pet=os.path.join(data_dcm_directory,'Patient2.PT._.3.144.2019.11.21.11.52.34.596.71220765.dcm')
# Read the file's meta-information without reading bulk pixel data
file_reader_pet=sitk.ImageFileReader()
file_reader_pet.SetFileName(file_name_pet)
file_reader_pet.ReadImageInformation()

# Get the sorted file names, open all files in the directory and read the meta-information without reading the bulk pixel data
series_ID_pet=file_reader_pet.GetMetaData('0020|000e')
sorted_file_names_pet=sitk.ImageSeriesReader.GetGDCMSeriesFileNames(data_dcm_directory,series_ID_pet)
print("Series ID corresponding to PET data:",series_ID_pet)

#### 1.2 Lecture des images et de leurs attributs

In [ ]:
# Read DICOM CT and PET images and convert them to NumPy arrays
# Access to their attributes (size, origin and spacing)
img_ct=sitk.ReadImage(sorted_file_names_ct)
print('Image CT size: ', img_ct.GetSize())
print('Image CT origin: ', img_ct.GetOrigin())
print('Image CT spacing: ', img_ct.GetSpacing(),'\n')

img_pet=sitk.ReadImage(sorted_file_names_pet)
print('Image PET size: ', img_pet.GetSize())
print('Image PET origin: ', img_pet.GetOrigin())
print('Image PET spacing: ', img_pet.GetSpacing())

### 2. Affichage des images DICOM

#### 2.1 Utilisation de Matplotlib et de la fonction interact

Comme dans le notebook précédent, on va d'abord convertir les images qui sont toujours des objets SimpleITK en arrays NumPy.

In [ ]:
nimg_ct = sitk.GetArrayFromImage(img_ct)
print(nimg_ct.shape)
nimg_pet = sitk.GetArrayFromImage(img_pet)
print(nimg_pet.shape)

Ci-dessous, on crée une fonction permettant l'affichage d'une image 3D selon ses 3 plans de coupe

In [ ]:
# Home-made function to show a 3D image in three views
def show_image(arr,sx,sy,sz,color):
    fig, ax = plt.subplots(1,3, figsize=(10, 10))
    ax[0].imshow(arr[sz,:,:], cmap=str(color)) #
    ax[0].set_title("Vue axiale (transverse)")
    ax[1].imshow(arr[:,sy,:], cmap=str(color), origin='lower')
    ax[1].set_title("Vue coronale (frontale)")
    ax[2].imshow(arr[:,:,sx], cmap=str(color), origin='lower')
    ax[2].set_title("Vue sagittale")

In [ ]:
# View CT
interact(show_image, arr=fixed(nimg_ct), sx=(0,img_ct.GetSize()[0]-1), sy=(0,img_ct.GetSize()[1]-1), sz=(0,img_ct.GetSize()[2]-1),color='gray');

In [ ]:
# View PET
interact(show_image, arr=fixed(nimg_pet), sx=(0,img_pet.GetSize()[0]-1), sy=(0,img_pet.GetSize()[1]-1), sz=(0,img_pet.GetSize()[2]-1),color='jet');

Comme vous pouvez le constater, l'utilisation de la fonction `imshow` de Matplotlib (très efficace par ailleurs) devient problématique dès lors qu'on travaille avec des images qui ont une haute dynamique ou qui ont un voxel size non isotrope, ce qui est le cas de la plupart des images médicales 3D. L'image apparait ainsi déformée à l'affichage.

Dans les cellules suivantes, on affiche les deux modalités d'imagerie superposées l'une sur l'autre.

In [ ]:
# Home-made function to overlay two 3D images from different modality imaging (here, CT and PET)
def show_fusion(arr, nslice, opacity=0.7 ):
    plt.imshow(nimg_pet[nslice,:,:], cmap=plt.cm.jet)
    plt.colorbar()
    a = arr[nslice,:,:]
    plt.imshow(a, alpha=opacity, cmap=plt.cm.gray)
    plt.colorbar()
    #plt.savefig('fusion.png',dpi=300,transparent=True,bbox_inches="tight")

In [ ]:
# View overlaid CT and PET images
interact(show_fusion, arr=fixed(nimg_ct), nslice=(0,len(nimg_pet)-1), opacity=(0,1,0.1));

----
**Question 2.** Quel est le problème et comment le résoudre ?

----

In [ ]:
# Assuming these two images are already registered, we can resample the CT volume according to the origin, spacing and dimensions of the PET volume.
# Hence, we preserve their individual locations as each has a different origin, spacing and orientation in the same space.
print("Transformation appliquée :",sitk.Transform())
img_resampled_ct = sitk.Resample(img_ct, img_pet, sitk.Transform(), sitk.sitkLinear, 0)
print("On peut vérifer les dimensions de l'image CT après rééchantillonnage\n")
print("Taille de l'image CT après rééchantillonnage : ", img_resampled_ct.GetSize())
print("Taille des pixels de l'image CT après rééchantillonnage :" , img_resampled_ct.GetSpacing())
nimg_resampled_ct = sitk.GetArrayFromImage(img_resampled_ct)
output_file_name_3D_resampled_ct = os.path.join(OUTPUT_DIR, '3DImageResampledCT.nii')
sitk.WriteImage(img_resampled_ct, output_file_name_3D_resampled_ct)

In [ ]:
# View overlaid resampled CT and PET images
interact(show_fusion, arr=fixed(nimg_resampled_ct), nslice=(0,len(nimg_pet)-1), opacity=(0,1,0.1));

### 2.2 Sauvegarder les series d'images DICOM en images directement 3D

In [ ]:
# Read the bulk pixel data and write them as a single 3D nii or mha file
output_file_name_3D_ct = os.path.join(OUTPUT_DIR, '3DImageCT.nii') #'3DImageCT.mha'
sitk.WriteImage(img_ct, output_file_name_3D_ct)
output_file_name_3D_pet = os.path.join(OUTPUT_DIR, '3DImagePET.nii') #'3DImagePET.mha'
sitk.WriteImage(img_pet, output_file_name_3D_pet)

### 2.3 Utilisation d'autres logiciels
Quand on travaille avec des images médicales, il est recommandé de les visualiser à l'aide de logiciels dédiés comme par exemple [3D Slicer](https://www.slicer.org/), [ITK-SNAP](http://www.itksnap.org/pmwiki/pmwiki.php) ou [ImageJ](http://imagej.nih.gov/ij/), tous téléchargeables gratuitement.
Téléchargez un de ces logiciels et ouvrez ```output_file_name_3D_pet``` ainsi que ```output_file_name_3D_ct```.



## Deuxième méthode pour charger des séries d'images DICOM

Dans les cellules suivantes, vous allez découvrir comment avec **une seule boucle**, on peut **lire**, **afficher** et **sauvegarder** des séries d'images DICOM (à réutiliser sans modération)

In [ ]:
data_dcm_directory = './data/patient2/'
print("Path to DICOM data directory:",data_dcm_directory)

# Directory contains multiple DICOM studies/series, store in dictionary with key being the series ID
reader = sitk.ImageSeriesReader()
series_file_names = {}
series_IDs = reader.GetGDCMSeriesIDs(data_dcm_directory)

In [ ]:
# Display both modality imaging side by side
plt.subplots(1,2,figsize=(10,5))
# Check that we have at least one series
if series_IDs:
    print("Il y a " + str(len(series_IDs)) + " serie(s) dans le dossier " + data_dcm_directory+'\n')
    for i, series in enumerate(series_IDs):
        print("Serie numéro", i+1)
        print('Series ID:', series)
        series_file_names[series] = reader.GetGDCMSeriesFileNames(data_dcm_directory, series)
        reader.SetFileNames(series_file_names[series])
        reader.MetaDataDictionaryArrayUpdateOn()
        reader.LoadPrivateTagsOn()
        img = reader.Execute()
        modality=reader.GetMetaData(2,'0008|0060')
        print('Modality:',modality)
        sitk.WriteImage(img,os.path.join(OUTPUT_DIR,str(modality)+".nii"))
        print("-> Sauvegarde de l'image "+str(modality)+".nii "+"-> OK\n")
        # Display the image slice from the middle of the stack, z axis
        z = int(img.GetDepth()/2)
        plt.subplot(1,2,i+1)
        if modality == 'CT':
            plt.imshow(sitk.GetArrayViewFromImage(img)[z,:,:], cmap='gray')
        if modality == 'PT':
            plt.imshow(sitk.GetArrayViewFromImage(img)[z,:,:], cmap='jet')
        plt.title('Modality: ' + modality)
        plt.axis('off')
        plt.colorbar(shrink=0.7)
else:
    print('Data directory does not contain any DICOM series.')

# C) Lecture des en-têtes DICOM à l'aide de la librairie pydicom

La librairie [pydicom](https://pydicom.github.io/) offre de nombreuses fonctionnalités pour manipuler assez simplement des fichiers DICOM en Python, en particulier leurs meta-données.

Dans le notebook précédent, nous avons ouvert et analysé le fichier image intitulé `PosteriorStatic001_DS.dcm`. L'extension de ce fichier nous indique qu'il s'agit d'une image DICOM et son nom qu'il s'agit certainement d'une image de scintigraphie. Nous allons vérifier cela et plus généralement nous intéresser aux meta-données de cette image en l'ouvrant cette fois-ci non plus avec `SimpleITK` mais avec avec le module `pydicom`.

Importez le module `pydicom` pour pouvoir l'utiliser dans votre programme.

In [ ]:
!pip install pydicom
import pydicom as dcm

La commande suivante execute la fonction `read_file` du module pydicom qu'on a renommé dcm.

In [ ]:
dataset = dcm.dcmread('./data/rein/PosteriorStatic001_DS.dcm')

En tapant `dataset`, on affiche l'ensemble des meta-données du fichier DICOM.

In [ ]:
dataset

* Les meta-données sont organisées en plusieurs **groupes d'informations**
* Chaque groupe d'informations contient plusieurs **éléments** ou **attributs**

L'ensemble de ces meta-données constitue un dictionnaire de données DICOM.

<center>
<img src="https://github.com/marsvn/PythonM2-jour2/blob/main/data/dicomstructure.png?raw=1">
</center>

| Groupe d'information     |     Signification  |
| ------------- |: -------------: |
| 0008 |  Meta-information (File Meta Information Version, SOP Class UID, SOP Instance UID, Transfert Syntax UID, ...)<br>Identification du centre (date d'examen, type d'examen, fabricant de la machine, hôpital, identification de la machine, ...) |
| 0010 | Informations sur le patient (nom, identification, date de naissance, sexe, ...) |
| 0018 | Informations sur l'acquisition de l'information (épaisseur de coupe, bolus, inclinaison du statif,<br> paramètres kV, temps d'écho, position du patient, etc...). |
| 0020 | Positionnement et informations relatives à l'acquisition (orientation du patient, série, plan de référence, <br>nombre d'images dans l'acquisition, Study Instance UID, Series Instance UID,...) |
| 0028 | Présentation de l'image (dimensions, niveaux de gris, tables de couleurs, bits alloués, <br>bits stockés, bit le plus significatif, WindowWidth, WindowCenter...) |
| 4000 | Texte |
| 7FE0 | Pixels de l'image (pixel data) |

Choisissons un attribut particulier

In [ ]:
dataset[0x008,0x060]

La commande précédente affiche la ligne correspondant à l'élément ou attribut ou *tag element* (0008, 0060).

Le format de cette ligne est le suivant :

- **(0008, 0060)** : numéro de l'attribut (*tag number*), codé en hexadecimal suivant (numéro du groupe, numéro de l'attribut)

- **Modality** : nom de cet attribut, s'il est connu

- **CS**: valeur de représentation de l'attribut, si elle est connue

- **'NM'**: valeur de l'attribut considéré


Pour accéder à un attribut spécifique, on peut utiliser son numéro (dans l'exemple ci-dessus, c'est (0x008,0x060)) ou le mot-clé DICOM correspondant (dans l'exemple ci-dessus, c'est `Modality`).

Le mot-clé est généralement le même que le nom de l'élément sans espace. Quelques exceptions néanmoins comme `(0010,0010) Patient’s Name` dont le mot-clé correspondant est `PatientName`.

Une liste de tous les mots-clé peut être consultée [ici](http://dicom.nema.org/medical/dicom/current/output/chtml/part06/chapter_6.html).

In [ ]:
dataset['Modality']

In [ ]:
dataset[0x008,0x060].value

In [ ]:
dataset.Modality

In [ ]:
print("Patient's Name:", dataset.PatientName)
print('Patient ID:', dataset[0x10,0x20].value)

La commande suivante permet de trier le dataset, en affichant uniquement l'ensemble des éléments du groupe d'informations `0010`

In [ ]:
dataset.group_dataset(0x0010)

La commande suivante permet d'afficher la valeur du tag number associé à l'attribut `Patient ID`.

In [ ]:
print(dataset.data_element("PatientID").tag)

Vérifions

In [ ]:
print(dataset[0x0010,0x0020].value)

La fonction `dataset.dir()` permet de retrouver tous les mots-clés contenant une chaîne de caractère donnée.

In [ ]:
dataset.dir("UID")

In [ ]:
liste = dataset.dir('patien')
print(liste)

In [ ]:
for i in liste:
    print(i,"=>",dataset[i].value)

## Utilisation de pydicom conjointement avec Matplotlib

L'utilisation de la bibliothèque pydicom est principalement dédiée à l'exploration et la manipulation des meta-données DICOM.

Il peut être utile de visualiser les données image.

Pour cela, plusieurs options :

* Utiliser un logiciel capable de lire le DICOM (3D Slicer, ImageJ, ITK-SNAP)
* Utiliser pydicom avec Matplotlib

L'attribut `(7FE0,0010) Pixel Data` donne accès aux données images brutes d'un fichier DICOM.

L'attribut `Dataset.pixel_array` retourne directement un tableau Numpy.


In [ ]:
pixel_bytes = dataset.PixelData

In [ ]:
pixel_bytes

In [ ]:
arr = dataset.pixel_array

In [ ]:
arr

In [ ]:
print(arr.shape, arr.size)

Taille de l'image : 128 x 128 (16394 pixels)

De cette façon, Matplotlib peut accéder à l'information contenue dans cette image 2D et l'afficher.

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(arr, cmap='jet')
plt.colorbar()

## Exercice 1
---

1. Retrouver les dimensions de l'image `PosteriorStatic001_DS.dcm` et sa résolution
2. Retrouver quel était le radiopharmaceutique injecté

In [ ]:
...

In [ ]:
print(dataset...)
print(dataset.data_element(...).tag)

In [ ]:
print(dataset[...,...][0][...,...][0][...,...].value)

## Modification de fichiers par lot

On peut aussi utiliser pydicom pour modifier certains tags d'une série de fichiers DICOM.

Une utilisation potentielle de cette fonctionnalité est l'**anonymisation** de fichiers.

Les fichiers présents dans le répertoire `data/patient2` sont déjà anonymisés mais on va à nouveau modifier l'attribut `PatientName` comme on pourrait le faire pour anonymiser ces fichiers.

In [ ]:
import pydicom as dcm
from pathlib import Path

# La syntaxe suivante permet de rechercher une chaîne de caractères précise dans un liste de fichiers
dcm_files = Path('./data/patient2/').rglob('*.dcm')
dcm_files_CT = Path('./data/patient2/').rglob('*CT*.dcm')
dcm_files_PT = Path('./data/patient2/').rglob('*PT*.dcm')

# La fonction len() affiche la longueur d'une liste (donc ici le nombre de fichiers)
print('Nombre de fichiers total dans la série :',len(list(dcm_files)))
print('Nombre de fichiers TDM dans la serie :',len(list(dcm_files_CT)))
print('Nombre de fichiers TEP dans la serie :',len(list(dcm_files_PT)))

In [ ]:
# Liste contenant tous les fichiers DICOM du dossier patient2
dcm_files = Path('./data/patient2/').rglob('*.dcm')

# Création du nouveau répertoire patient2_anonymised s'il n'existe pas (s'il existe, on ne fait rien)
output_dir = Path('./data/patient2_anonymised/')
output_dir.mkdir(exist_ok=True)

# Boucle sur l'ensemble des fichiers DICOM du répertoire patient2
for file_dcm in dcm_files:
    # Lecture de l'entête DICOM de chaque fichier
    ds = dcm.dcmread(str(file_dcm))
    #print(ds.PatientName)
    # Les images DICOM de sortie (après anonymisation) auront le même nom
    out_file = file_dcm.name
    # Modification de l'attribut PatientName : patient2 --> M2
    ds.PatientName = "M2"
    # Réécriture des nouveaux fichiers DICOM et stockage dans le dossier patient2_anonymised
    out_file = output_dir / file_dcm.name
    ds.save_as(out_file)

In [ ]:
# On vérifie que l'attribut PatientName a bien été modifié
newdcm_files = Path('./data/patient2_anonymised/').rglob('*.dcm')
# Nom du patient après anonymisation
print("Vérification --> nom du patient après anonymisation :",dcm.dcmread(str(list(newdcm_files)[0])).PatientName)

## Exercice 2
---

Objectifs de l'exercice :

1. Lecture d'images TEP et CT préalablement sauvegardées au format DICOM
2. Identification de la modalité TEP
3. Ajout d'une étape de calcul du facteur de conversion en unités SUV
4. Sauvegarde du fichier SUV
5. Calcul d'un volume tumoral
6. Evaluation de la charge métabolique

Pour cela, il est nécessaire d'accéder aux valeurs des attributs DICOM suivants :

* `Series Time`: Time the series started. Series Date and Series Time are used as the reference time for all PET Image Attributes that are temporally related, including activity measurements.
* `Radiopharmaceutical Start Time`: Time of start of administration. The actual time of radiopharmaceutical administration to the patient for imaging purposes, using the same time base as Series Time. Example: "070907.0705" represents a time of 7 hours, 9 minutes and 7.0705 seconds.
* `Radionuclide Total Dose`: The radiopharmaceutical dose administered to the patient measured in Becquerels (Bq) at the Radiopharmaceutical Start Time.
* `Radionuclide Half Life`: The radionuclide half life, in seconds, to be used in the correction of this image.
* `Patient's Weight`: Weight of the Patient, in kilograms.

Le site internet https://dicom.innolitics.com/ciods/pet-image regroupe l'ensemble des attributs DICOM relatif à la modalité d'imagerie TEP et renseigne les valeurs de tag number correspondants.

## Rappels sur le SUV (Standardized Uptake Value)

La valeur de fixation normalisée (SUV en anglais pour Standardized Uptake Value) est l'index semi-quantitatif le plus communément utilisé en routine clinique pour caractériser la fixation du FDG en TEP.

Dans  sa  définition  la  plus  classique,  le  SUV  se  calcule comme   le   rapport   de   la   concentration moyenne du radiotraceur dans   un   tissu   d’intérêt (en Bq/mL) à un instant donné à l'activité totale injectée au patient (en Bq), rapporté à son volume (en mL). En supposant que le patient a une masse volumique de 1 g/mL, la valeur du volume en mL est la même que celle de la masse en grammes.

Le SUV est calculé en pratique en rapportant l'activité injectée au poids du patient.

$$ SUV_{BW}=\frac{[^{18}F-FDG]}{\frac{A_{inj}(t_{acq})}{W}} $$

Bien sûr, il faut tenir compte dans la formule ci-dessus du délai de temps $\Delta t$ entre l'injection et l'acquisition.

$$ A_{inj_{c}}=A_{inj}(t_{acq})\times 2^{(\frac{-\Delta t}{T_{1/2}})} $$

Le SUV est donc une quantité sans dimension, dont l’usage fréquent est en particulier dû à la simplicité de sa méthode de calcul.
* Si le radiotraceur se répartissait uniformément dans tout l’organisme,   le   SUV   serait   identique   en   tout   point   de l’organisme  et  égal  à 1.Tout  écart  du  SUV à 1  traduit  une répartition non uniforme du radiotraceur dans le volume dans lequel il s’est distribué.
* Le pouvoir de normalisation du SUV facilite l’interprétation   semi-quantitative   des   images.   En   effet,   même   si   la normalisation sous-jacente au calcul du SUV est grossière, elle est   tout   de   même   largement   préférable   à l’absence   de normalisation.  
* L’expression  des  images  en  échelle  SUV  rend comparables des images acquises chez différents patients, et des images acquises chez un patient à différents stades de la prise en charge thérapeutique.
* La normalisation n’est certes pas parfaite mais elle réduit tout de même considérablement la variabilité inter-patients et inter-examens.  
* Le SUV reste donc un index simpliste pour caractériser le métabolisme du glucose.

NB. Cette formule suppose que les images TEP ont été préalablement calibrées (conversion des valeurs d'intensité en activité). On peut le vérifier à partir de l'attribut `Units`. Dans le cas contraire, il faudrait ajouter une conversion supplémentaire :

$PET_{activity} = PET_{raw} \times rescale\_ slope + rescale\_intercept$

$rescale\_ slope$ et $rescale\_intercept$ sont deux attributs DICOM accessibles (numéro (0028,1052) et (0028,1053) respectivement).

In [ ]:
import pydicom

data_dir = './data/patient4'

# Define current working directory
pwd_dir = os.getcwd()

# Define target directory
target_dir = '/content/PythonM2-jour2/output'

# Liste de toutes les series DICOM du répertoire source
series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(...)

print("Il y a " + str(len(series_IDs)) + " série(s) présentes dans le répertoire " + data_dir)
print(*series_IDs,sep='\n')

for i, serie in enumerate(series_IDs): # Pour chaque série du répertoire source
    print(str(i) + "...",end = '')
    reader=sitk.ImageSeriesReader() # Définition du reader
    # Liste de tous les fichiers concernant la série considérée
    series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(...,...)
    reader.SetFileNames(...)
    # On va récupérer tous les tags DICOM de la série en lisant le 1er fichier de la série
    ds = ...
    print(ds[0x0008,0x0060].value,end='-')
    print(ds.Modality,end='|')
    print(ds.pixel_array.itemsize,end='|')
    print(ds.ProcedureCodeSequence[0][0x0008,0x0104].value,end='')
    if ds.pixel_array.itemsize!=1: # itemsize() function return the length of one array element in bytes (octets)
        try:                       # 1 <=> (u)int8 et 2 <=> (u)int16
            IMAGE=reader.Execute()
            chaine=str(i)+"-"+ds.Modality+".nii"
            sitk.WriteImage(IMAGE,os.path.join(target_dir,chaine))
            print("->"+chaine+"-> OK")
            #break
        except:
            print("->fail")
        verif_flag = True
        if ... == "PT":
            # Calcul du scale factor permettant la conversion Bq/mL<=>SUV
            RST = ... # RadiopharmaceuticalStartTime
            RST = ... # string to numerical value in sec
            RTD = ... # RadionuclideTotalDose [Bq]
            RHL = ... # RadionuclideHalfLife [s]
            ST  = ...                   # SeriesTime
            ST  = ... # string to numerical value in sec
            PW  = ...                  # PatientWeight [kg]
            # RSD=(ds[0x0054,0x0016][0][0x0018,0x1078].value) #
            CD = ...            # CorrectedDose
            PET_SUV_ScaleFactor = ... # On multiplie par 1000 pour avoir le poids en g
            print("PET_SUV_ScaleFactor = ",PET_SUV_ScaleFactor)
            if verif_flag:
                print("Series Time       : " + (...)
                print("Dose              : " + str(...) + " Bq ou " + str(.../1e6) + " MBq")
                print("Patient's Weight  : " + str(...)+" kg")
                print("Half-life         : " + str(...) + " sec ou " + str(.../60) + " min")
                print("Start Time        : " + ...)
    else:
        print()
print("Fin")

In [ ]:
# Sauvegarde de l'image TEP après conversion en unités SUV
sitk.WriteImage(...)

 (Option) Ouvrir le volume TEP avec ITK-SNAP et trouver manuellement la coupe passant par la tumeur pulmonaire

 Ici on vous donne les coordonnées du centre de la tumeur: **(89,131,329)**.<br>
**<span style='color:red'>Attention, les index commencent à zéro dans Python mais ... à 1 dans ITK-SNAP</span>**.

On trouve pour ce voxel 5892 Bq/mL dans ITK-SNAP

In [ ]:
print('Intensité du pixel :', IMAGE[89,131,329])

Croppons le volume TEP à +/- 10 coupes dans chaque direction (pour ne pas etre gêné par l'activité cardiaque qui est intense chez ce patient).

En réalité, on ne va pas réellement "rogner" le volume, mais on va mettre à 0 toutes les valeurs en dehors de cette "boite englobante" définie autour de la tumeur (ce qui permettra de faire un masque de la tumeur par la suite)

In [ ]:
tumor_center = [90,130,337] # Définition du centre de la tumeur
nb_slices_to_crop = 10      # Demi-taille de la boîte englobante
TC , NS = tumor_center, nb_slices_to_crop
# Pour faciliter la lecture des lignes suivantes, on crée un nouveau volume 'croped'
IMAGE_cropped = IMAGE   # Sur la base du volume TEP initial
IMAGE_cropped_np = sitk.GetArrayFromImage(IMAGE_cropped)  # Copie du volume TEP initial

# On va mettre à 0 ce qui est en dehors de notre boîte englobante
# Attention à l'indexation des arrays NumPy dont l'ordre est inversé par rapport à l'image SimpleITK de départ
IMAGE_cropped_np[:,:,TC[0] + NS:] = 0
IMAGE_cropped_np[:,:,:TC[0] - NS] = 0

IMAGE_cropped_np[:,TC[1] + NS:,:] = 0
IMAGE_cropped_np[:,:TC[1] - NS,:] = 0

IMAGE_cropped_np[TC[2] + NS:,:,:] = 0
IMAGE_cropped_np[:TC[2] - NS,:,:] = 0

plt.subplots(1,2,figsize=(10,5))
plt.subplot(121)
plt.imshow(sitk.GetArrayFromImage(IMAGE)[TC[2],:,:],cmap='jet')
plt.title('Image originale')
plt.axis('off')
plt.subplot(122)
plt.imshow(IMAGE_cropped_np[TC[2],:,:],cmap='jet')
plt.title('Image croppée')
plt.axis('off')

Récupération de la valeur du pixel d'intensité maximale en Bq/mL et conversion en unités SUV

In [ ]:
total_flag = False     # Sélectionne si on travaillera sur le volume TEP entier (total_flag=True)
                       # ou si on travaillera sur le volume rogné (total_flag=False)

if total_flag:
    print("Mesures obtenues en utilisant tout le volume TEP")
    IMAGE_np = sitk.GetArrayFromImage(IMAGE)
else:
    print("Mesures obtenues en utilisant le volume TEP rogné (cropped)")
    IMAGE_np = IMAGE_cropped_np

IMAGE_np_max = IMAGE_np.max()
print("Valeur d'intensité maximale : " + str(IMAGE_np_max) + " Bq/mL")

# Conversion Bq/mL<->SUV

IMAGE_np_max_SUV = IMAGE_np_max * PET_SUV_ScaleFactor
print("Voxel le + intense : " + str(IMAGE_np_max) + " Bq/mL <=> " + \
      str(IMAGE_np_max_SUV) + " SUVmax")

(Option) Vérifier cette valeur en ouvrant le volume 1-PT.nii dans ITK-SNAP.
En utilisant l'inspecteur (menu Tools>Image Contrast>Contrast Adjustement...  ou [Ctrl]+[I] puis l'onglet contraste qui donne la valeur maximale (en Bq/mL)

Seuillage de l'image à 15 ou 40% de cette valeur max et sauvegarde du masque binaire obtenu

In [ ]:
seuil = 0.15

IMAGE_np_seuil = seuil * IMAGE_np.max()    # Calcul du seuil relatif pour cette image
SEG_size = IMAGE.GetSize()                 # Récupère la taille de l'image, ce sera la taille de l'image segmentation
SEG = sitk.Image(SEG_size,sitk.sitkUInt8)  # Crée une image segmentation (masque binaire) 8 bits suffisent
SEG.CopyInformation(IMAGE)                 # Copie pixel_size, spacing , etc

SEG_mask = IMAGE_np >= IMAGE_np_seuil  # Masque qui correspond a notre condition de segmentation
plt.imshow(SEG_mask[TC[2],:,:],cmap='gray')
plt.axis('off')
print("Valeur max   : ",IMAGE_np.max()," Bq/mL")
print("Valeur seuil : ",IMAGE_np_seuil, "Bq/mL ",end = "")
print(" <=> ",IMAGE_np_seuil * PET_SUV_ScaleFactor," SUV")

# Créer l'objet sitk de segmentation a partir du masque (que l'on multiplie a une matrice de 'ones')
SEG = sitk.GetImageFromArray(np.multiply(SEG_mask,np.ones(SEG_mask.shape)))
# Sauvegarder la segmentation dans un fichier SEG.nii
sitk.WriteImage(SEG,os.path.join(target_dir,"SEG.nii"))


* Essayer d'autres seuils de segmentation [0;1] (0,40 est très sélectif, tenter 0,15 pour le volume rogne et 0,05 pour le volume entier)

* Essayer sur l'image TEP entiere et sur l'image rognée (mettre total_flag = False)

*  (Option) Ne pas oublier dans le quart inferieur gauche d'ITK-SNAP de cliquer sur 'update' pour visualiser la segmentation en 3D

**Calcul du volume tumoral correspondant**

On fait le calcul en nombre de voxels puis en mm3

In [ ]:
# Calcul de la taille du voxel permettant la conversion vox<->mm<sup>3</sup>

print('Taille du pixel (mm) : :', [IMAGE.GetSpacing()[0],IMAGE.GetSpacing()[1]])
print('Epaisseur de coupe (mm) :', IMAGE.GetSpacing()[2])

# Calcul du volume (en mm3) du voxel directement
vox2vol_Factor = ...

print("Facteur de conversion voxel<->mm3 = " + str(vox2vol_Factor) + " (mm3/voxel)")

In [ ]:
seg_nb_vox = len(SEG_mask[SEG_mask==1])
seg_nb_mm3 = seg_nb_vox * vox2vol_Factor
print("Seuil :",str(int(100*seuil))+" %")
print(seg_nb_vox,"voxels de segmentation /",len(SEG),"voxels")
print(str(seg_nb_vox) + " voxels <=> " + str(seg_nb_mm3) + " mm3")

**Evaluation de la charge métabolique**

On utilise la somme cumulée des valeurs des voxels du volume segmenté

In [ ]:
seg_cumsum = ...
print("Total cumulé    = " , seg_cumsum)
print("Moyenne         = " , seg_cumsum/seg_nb_vox)

(Option) Comparez cette valeur à la valeur donnée dans ITK-SNAP (menu
Segmentation>Volumes and Statistics...): `/usr/local/itksnap-3.8.0-20190612-Linux-gcc64/bin/itksnap -g "1-PT.nii  -s SEG.nii`

# Conclusion

- PyDicom est plus puissant pour la gestion des métadonnées (permet de les modifier et les sauvegarder).

- SimpleITK limite l'accès aux métadonnées car son objectif principal est de faire du traitement d'images.